<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/332_SalesEnab_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lead Prioritization Utilities for Sales Enablement Orchestrator

In [ ]:
"""Lead Prioritization Utilities for Sales Enablement Orchestrator

This module contains utilities for prioritizing and ranking leads.
Uses toolshed.prioritization for scoring, with lead-specific logic.
"""

from typing import Dict, List, Any, Optional
from toolshed.prioritization import calculate_priority_score, normalize_severity


def normalize_urgency(urgency: str) -> float:
    """
    Convert urgency string to numeric score.

    Args:
        urgency: "low" | "medium" | "high"

    Returns:
        Numeric score (0.0 - 100.0)
    """
    urgency_map = {
        "low": 30.0,
        "medium": 60.0,
        "high": 100.0
    }
    return urgency_map.get(urgency.lower(), 50.0)


def normalize_budget_range(budget_range: str) -> float:
    """
    Convert budget range string to numeric score.

    Args:
        budget_range: e.g., "10k-25k", "50k-100k", "250k+"

    Returns:
        Numeric score (0.0 - 100.0) based on budget size
    """
    budget_map = {
        "10k-25k": 30.0,
        "25k-50k": 50.0,
        "50k-100k": 70.0,
        "100k-250k": 85.0,
        "250k+": 100.0
    }
    return budget_map.get(budget_range, 50.0)


def calculate_lead_priority_score(
    lead: Dict[str, Any],
    signal: Optional[Dict[str, Any]] = None,
    weights: Optional[Dict[str, float]] = None
) -> float:
    """
    Calculate priority score for a lead.

    Uses weighted scoring based on:
    - intent_score (from lead)
    - engagement_score (from signal)
    - deal_risk_score (inverse, from signal)
    - budget_range (from lead)
    - urgency (from signal)

    Args:
        lead: Lead dictionary
        signal: Optional signal dictionary (if not provided, uses defaults)
        weights: Optional scoring weights (defaults from config)

    Returns:
        Priority score (0.0 - 100.0+)
    """
    if weights is None:
        weights = {
            "intent_score": 0.30,
            "engagement_score": 0.25,
            "deal_risk_score": 0.20,  # Inverse - lower risk = higher score
            "budget_range": 0.15,
            "urgency": 0.10
        }

    # Get values from lead and signal
    intent_score = lead.get("intent_score", 0.0) * 100.0  # Convert 0-1 to 0-100

    if signal:
        engagement_score = signal.get("engagement_score", 0.0) * 100.0
        deal_risk_score = signal.get("deal_risk_score", 0.5) * 100.0
        urgency = signal.get("urgency", "medium")
    else:
        # Defaults if no signal
        engagement_score = 50.0
        deal_risk_score = 50.0
        urgency = "medium"

    budget_range = lead.get("budget_range", "25k-50k")

    # Build issue dict for toolshed prioritization
    issue = {
        "intent_score": intent_score,
        "engagement_score": engagement_score,
        "deal_risk_score": 100.0 - deal_risk_score,  # Inverse: lower risk = higher score
        "budget_range": normalize_budget_range(budget_range),
        "urgency": normalize_urgency(urgency)
    }

    # Use toolshed prioritization
    priority_score = calculate_priority_score(
        issue=issue,
        context={},
        weights=weights,
        field_mappers={}
    )

    return priority_score


def prioritize_leads(
    leads: List[Dict[str, Any]],
    signals_lookup: Dict[str, Dict[str, Any]],
    weights: Optional[Dict[str, float]] = None
) -> List[Dict[str, Any]]:
    """
    Prioritize all leads by calculating priority scores.

    Args:
        leads: List of lead dictionaries
        signals_lookup: Dictionary mapping lead_id to signal dict
        weights: Optional scoring weights

    Returns:
        List of leads with priority_score added, sorted by priority (descending)
    """
    prioritized = []

    for lead in leads:
        lead_id = lead.get("lead_id")
        signal = signals_lookup.get(lead_id) if lead_id else None

        priority_score = calculate_lead_priority_score(lead, signal, weights)

        prioritized_lead = lead.copy()
        prioritized_lead["priority_score"] = priority_score

        # Add signal data if available
        if signal:
            prioritized_lead["engagement_score"] = signal.get("engagement_score", 0.0)
            prioritized_lead["deal_risk_score"] = signal.get("deal_risk_score", 0.5)
            prioritized_lead["urgency"] = signal.get("urgency", "medium")
            prioritized_lead["recommended_action"] = signal.get("recommended_action", "prioritize outreach")
        else:
            prioritized_lead["engagement_score"] = 0.0
            prioritized_lead["deal_risk_score"] = 0.5
            prioritized_lead["urgency"] = "medium"
            prioritized_lead["recommended_action"] = "assign rep and initiate outreach"

        prioritized.append(prioritized_lead)

    # Sort by priority_score (descending)
    prioritized.sort(key=lambda x: x.get("priority_score", 0.0), reverse=True)

    return prioritized


def get_top_priority_leads(
    prioritized_leads: List[Dict[str, Any]],
    top_n: int = 10
) -> List[Dict[str, Any]]:
    """
    Get top N priority leads.

    Args:
        prioritized_leads: List of prioritized leads (already sorted)
        top_n: Number of top leads to return

    Returns:
        Top N leads
    """
    return prioritized_leads[:top_n]



# Customer Needs Analysis Utilities for Sales Enablement Orchestrator

In [ ]:
"""Customer Needs Analysis Utilities for Sales Enablement Orchestrator

This module contains utilities for analyzing customer needs, pain points, and buying signals.
"""

from typing import Dict, List, Any, Optional
from datetime import datetime, timedelta


def analyze_customer_needs(
    lead: Dict[str, Any],
    interactions: Optional[List[Dict[str, Any]]] = None,
    deal: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Analyze customer needs, pain points, and buying signals.

    Args:
        lead: Lead dictionary
        interactions: Optional list of interactions for this lead
        deal: Optional deal dictionary for this lead

    Returns:
        Dictionary with needs analysis:
        {
            "lead_id": str,
            "pain_points": List[str],
            "budget_range": str,
            "buying_signals": List[str],
            "objection_likelihood": Dict[str, float],
            "product_fit": float,
            "recommended_approach": str
        }
    """
    lead_id = lead.get("lead_id")
    pain_points = lead.get("pain_points", [])
    budget_range = lead.get("budget_range", "25k-50k")

    # Extract buying signals from interactions
    buying_signals = []
    if interactions:
        for interaction in interactions:
            key_topics = interaction.get("key_topics", [])
            outcome = interaction.get("outcome", "")

            # Detect buying signals
            if "pricing" in key_topics:
                buying_signals.append("pricing discussed")
            if "demo" in outcome.lower() or "demo" in str(key_topics).lower():
                buying_signals.append("demo requested")
            if "proposal" in outcome.lower() or "proposal" in str(key_topics).lower():
                buying_signals.append("proposal requested")
            if interaction.get("sentiment") == "positive":
                buying_signals.append("positive engagement")

    # Calculate objection likelihood based on interactions and deal stage
    objection_likelihood = {
        "pricing": 0.3,  # Default
        "timeline": 0.2,  # Default
        "budget": 0.2,  # Default
        "competition": 0.1  # Default
    }

    if interactions:
        for interaction in interactions:
            key_topics = interaction.get("key_topics", [])
            sentiment = interaction.get("sentiment", "neutral")

            if "pricing" in key_topics:
                if sentiment == "negative":
                    objection_likelihood["pricing"] = 0.7
                else:
                    objection_likelihood["pricing"] = 0.5

            if "timeline" in key_topics or "implementation" in str(key_topics).lower():
                objection_likelihood["timeline"] = 0.4

            if "budget" in key_topics:
                objection_likelihood["budget"] = 0.5

    if deal:
        # Adjust objection likelihood based on deal stage and risk flags
        risk_flags = deal.get("risk_flags", [])
        if "pricing sensitivity" in risk_flags:
            objection_likelihood["pricing"] = 0.8
        if "budget uncertainty" in risk_flags:
            objection_likelihood["budget"] = 0.7

        stage = deal.get("stage", "")
        if stage == "Negotiation":
            objection_likelihood["pricing"] = max(objection_likelihood["pricing"], 0.6)

    # Calculate product fit (simple heuristic based on pain points and intent)
    intent_score = lead.get("intent_score", 0.5)
    product_fit = intent_score * 0.7 + (len(buying_signals) / 5.0) * 0.3
    product_fit = min(1.0, product_fit)  # Cap at 1.0

    # Generate recommended approach
    recommended_approach = _generate_recommended_approach(
        pain_points, buying_signals, objection_likelihood, product_fit
    )

    return {
        "lead_id": lead_id,
        "pain_points": pain_points,
        "budget_range": budget_range,
        "buying_signals": list(set(buying_signals)),  # Remove duplicates
        "objection_likelihood": objection_likelihood,
        "product_fit": round(product_fit, 2),
        "recommended_approach": recommended_approach
    }


def _generate_recommended_approach(
    pain_points: List[str],
    buying_signals: List[str],
    objection_likelihood: Dict[str, float],
    product_fit: float
) -> str:
    """
    Generate recommended sales approach based on analysis.

    This is rule-based for MVP. Can be enhanced with LLM later.
    """
    approach_parts = []

    # High product fit
    if product_fit > 0.7:
        approach_parts.append("Strong product fit - focus on ROI and automation benefits")
    elif product_fit > 0.5:
        approach_parts.append("Moderate product fit - emphasize pain point solutions")
    else:
        approach_parts.append("Lower product fit - focus on discovery and education")

    # Pricing objections likely
    if objection_likelihood.get("pricing", 0) > 0.6:
        approach_parts.append("Prepare pricing justification and ROI calculator")

    # Timeline concerns
    if objection_likelihood.get("timeline", 0) > 0.4:
        approach_parts.append("Address implementation timeline and provide case studies")

    # Strong buying signals
    if len(buying_signals) >= 2:
        approach_parts.append("Multiple buying signals detected - advance to proposal")

    # Pain points focus
    if pain_points:
        primary_pain = pain_points[0] if pain_points else ""
        approach_parts.append(f"Lead with solution for: {primary_pain}")

    return " | ".join(approach_parts) if approach_parts else "Standard outreach approach"


def analyze_all_customer_needs(
    leads: List[Dict[str, Any]],
    interactions_lookup: Dict[str, List[Dict[str, Any]]],
    deals_lookup: Dict[str, Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Analyze customer needs for all leads.

    Args:
        leads: List of lead dictionaries
        interactions_lookup: Dictionary mapping lead_id to list of interactions
        deals_lookup: Dictionary mapping lead_id to deal dict

    Returns:
        List of customer needs analysis dictionaries
    """
    analyses = []

    for lead in leads:
        lead_id = lead.get("lead_id")
        interactions = interactions_lookup.get(lead_id, [])
        deal = deals_lookup.get(lead_id)

        analysis = analyze_customer_needs(lead, interactions, deal)
        analyses.append(analysis)

    return analyses



# Test Phase 3: Lead Prioritization & Customer Analysis Utilities

In [ ]:
"""Test Phase 3: Lead Prioritization & Customer Analysis Utilities

Test utilities independently before building nodes.
"""

import sys
from pathlib import Path

# Add project root to path
project_root = Path(__file__).parent
sys.path.insert(0, str(project_root))

from agents.sales_enablement.utilities.lead_prioritization import (
    normalize_urgency,
    normalize_budget_range,
    calculate_lead_priority_score,
    prioritize_leads,
    get_top_priority_leads
)
from agents.sales_enablement.utilities.customer_analysis import (
    analyze_customer_needs,
    analyze_all_customer_needs
)
from agents.sales_enablement.utilities.data_loading import (
    load_leads,
    load_interactions,
    load_deals,
    load_signals,
    build_interactions_lookup,
    build_deals_lookup,
    build_signals_lookup
)
from config import SalesEnablementOrchestratorConfig


def test_normalize_urgency():
    """Test urgency normalization"""
    print("Testing normalize_urgency...")

    assert normalize_urgency("low") == 30.0
    assert normalize_urgency("medium") == 60.0
    assert normalize_urgency("high") == 100.0
    assert normalize_urgency("unknown") == 50.0  # Default

    print("✅ normalize_urgency test passed!")
    print()


def test_normalize_budget_range():
    """Test budget range normalization"""
    print("Testing normalize_budget_range...")

    assert normalize_budget_range("10k-25k") == 30.0
    assert normalize_budget_range("50k-100k") == 70.0
    assert normalize_budget_range("250k+") == 100.0
    assert normalize_budget_range("unknown") == 50.0  # Default

    print("✅ normalize_budget_range test passed!")
    print()


def test_calculate_lead_priority_score():
    """Test lead priority score calculation"""
    print("Testing calculate_lead_priority_score...")

    lead = {
        "lead_id": "L-001",
        "intent_score": 0.82,
        "budget_range": "50k-100k"
    }

    signal = {
        "lead_id": "L-001",
        "engagement_score": 0.76,
        "deal_risk_score": 0.28,
        "urgency": "high"
    }

    score = calculate_lead_priority_score(lead, signal)

    assert isinstance(score, float), "score should be float"
    assert score > 0, "score should be positive"
    assert score <= 200.0, "score should be reasonable (weighted sum)"

    print(f"✅ calculate_lead_priority_score test passed!")
    print(f"   Score: {score:.2f}")
    print()


def test_prioritize_leads():
    """Test prioritizing all leads"""
    print("Testing prioritize_leads...")

    config = SalesEnablementOrchestratorConfig()
    leads = load_leads(config.data_dir, config.leads_file)
    signals = load_signals(config.data_dir, config.signals_file)
    signals_lookup = build_signals_lookup(signals)

    prioritized = prioritize_leads(leads, signals_lookup)

    assert len(prioritized) == len(leads), "should have same number of leads"
    assert "priority_score" in prioritized[0], "should have priority_score"
    assert prioritized[0]["priority_score"] >= prioritized[-1]["priority_score"], "should be sorted descending"

    print(f"✅ prioritize_leads test passed!")
    print(f"   Prioritized {len(prioritized)} leads")
    print(f"   Top lead: {prioritized[0]['lead_id']} - Score: {prioritized[0]['priority_score']:.2f}")
    print(f"   Bottom lead: {prioritized[-1]['lead_id']} - Score: {prioritized[-1]['priority_score']:.2f}")
    print()


def test_get_top_priority_leads():
    """Test getting top N leads"""
    print("Testing get_top_priority_leads...")

    config = SalesEnablementOrchestratorConfig()
    leads = load_leads(config.data_dir, config.leads_file)
    signals = load_signals(config.data_dir, config.signals_file)
    signals_lookup = build_signals_lookup(signals)

    prioritized = prioritize_leads(leads, signals_lookup)
    top_5 = get_top_priority_leads(prioritized, top_n=5)

    assert len(top_5) == 5, "should return top 5"
    assert top_5[0]["priority_score"] >= top_5[-1]["priority_score"], "should be sorted"

    print(f"✅ get_top_priority_leads test passed!")
    print(f"   Top 5 leads:")
    for i, lead in enumerate(top_5, 1):
        print(f"     {i}. {lead['lead_id']} - {lead['company_name']} (Score: {lead['priority_score']:.2f})")
    print()


def test_analyze_customer_needs():
    """Test customer needs analysis"""
    print("Testing analyze_customer_needs...")

    lead = {
        "lead_id": "L-001",
        "pain_points": ["manual reporting", "forecast inaccuracy"],
        "budget_range": "50k-100k",
        "intent_score": 0.82
    }

    interactions = [
        {
            "interaction_id": "INT-001",
            "key_topics": ["pricing", "implementation timeline"],
            "sentiment": "positive",
            "outcome": "proposal_requested"
        }
    ]

    analysis = analyze_customer_needs(lead, interactions)

    assert "lead_id" in analysis, "should have lead_id"
    assert "pain_points" in analysis, "should have pain_points"
    assert "buying_signals" in analysis, "should have buying_signals"
    assert "objection_likelihood" in analysis, "should have objection_likelihood"
    assert "product_fit" in analysis, "should have product_fit"
    assert "recommended_approach" in analysis, "should have recommended_approach"

    assert len(analysis["buying_signals"]) > 0, "should detect buying signals"
    assert analysis["objection_likelihood"]["pricing"] > 0, "should calculate objection likelihood"

    print(f"✅ analyze_customer_needs test passed!")
    print(f"   Lead: {analysis['lead_id']}")
    print(f"   Buying signals: {analysis['buying_signals']}")
    print(f"   Product fit: {analysis['product_fit']}")
    print(f"   Approach: {analysis['recommended_approach'][:60]}...")
    print()


def test_analyze_all_customer_needs():
    """Test analyzing needs for all leads"""
    print("Testing analyze_all_customer_needs...")

    config = SalesEnablementOrchestratorConfig()
    leads = load_leads(config.data_dir, config.leads_file)
    interactions = load_interactions(config.data_dir, config.interactions_file)
    deals = load_deals(config.data_dir, config.deals_file)

    interactions_lookup = build_interactions_lookup(interactions)
    deals_lookup = build_deals_lookup(deals)

    analyses = analyze_all_customer_needs(leads, interactions_lookup, deals_lookup)

    assert len(analyses) == len(leads), "should analyze all leads"
    assert "lead_id" in analyses[0], "should have lead_id"
    assert "buying_signals" in analyses[0], "should have buying_signals"

    print(f"✅ analyze_all_customer_needs test passed!")
    print(f"   Analyzed {len(analyses)} leads")

    # Show sample analysis
    sample = analyses[0]
    print(f"   Sample: {sample['lead_id']}")
    print(f"     - Buying signals: {len(sample['buying_signals'])}")
    print(f"     - Product fit: {sample['product_fit']}")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 3: Lead Prioritization & Customer Analysis - Test Suite")
    print("=" * 60)
    print()

    test_normalize_urgency()
    test_normalize_budget_range()
    test_calculate_lead_priority_score()
    test_prioritize_leads()
    test_get_top_priority_leads()
    test_analyze_customer_needs()
    test_analyze_all_customer_needs()

    print("=" * 60)
    print("✅ All Phase 3 utility tests passed!")
    print("=" * 60)



# Sales Enablement Orchestrator Nodes

In [ ]:
"""Sales Enablement Orchestrator Nodes

This module contains all nodes for the Sales Enablement Orchestrator workflow.
Following the MVP-first approach: all nodes are rule-based (no LLM) for Phase 1-7.
"""

import json
from typing import Dict, Any
from config import SalesEnablementOrchestratorState, SalesEnablementOrchestratorConfig
from agents.sales_enablement.utilities.data_loading import (
    load_leads,
    load_sales_reps,
    load_interactions,
    load_deals,
    load_signals,
    build_leads_lookup,
    build_reps_lookup,
    build_interactions_lookup,
    build_deals_lookup,
    build_signals_lookup
)
from agents.sales_enablement.utilities.lead_prioritization import (
    prioritize_leads,
    get_top_priority_leads
)
from agents.sales_enablement.utilities.customer_analysis import (
    analyze_all_customer_needs
)


def goal_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for the Sales Enablement Orchestrator.

    This is a simple rule-based goal definition that sets the framework.
    No dependencies - this is the entry point.
    """
    lead_id = state.get("lead_id")
    rep_id = state.get("rep_id")
    focus_area = state.get("focus_area")

    # Build focus areas based on input
    if focus_area:
        focus_areas = [focus_area]
    else:
        focus_areas = [
            "lead_prioritization",
            "customer_needs_analysis",
            "outreach_generation",
            "follow_up_coordination",
            "rep_nudging",
            "deal_insights",
            "historical_insights",
            "executive_reporting"
        ]

    goal = {
        "objective": "Enable sales team performance by prioritizing leads, analyzing customer needs, generating outreach, coordinating follow-ups, nudging reps, and surfacing actionable insights",
        "lead_id": lead_id,
        "rep_id": rep_id,
        "focus_areas": focus_areas,
        "scope": "all_leads" if not lead_id else "single_lead",
        "scope_reps": "all_reps" if not rep_id else "single_rep"
    }

    return {
        "goal": goal,
        "errors": state.get("errors", [])
    }


def planning_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.
    Only depends on goal_node.
    """
    goal = state.get("goal")

    if not goal:
        return {
            "errors": state.get("errors", []) + ["planning_node: goal is required"]
        }

    focus_areas = goal.get("focus_areas", [])

    # Build plan based on focus areas
    plan = []
    step_num = 1

    # Always start with data loading
    plan.append({
        "step": step_num,
        "name": "data_loading",
        "description": "Load all data files (leads, reps, interactions, deals, signals) and create lookup dictionaries",
        "dependencies": [],
        "outputs": ["leads", "sales_reps", "interactions", "deals", "signals", "leads_lookup", "reps_lookup", "interactions_lookup", "deals_lookup", "signals_lookup"]
    })
    step_num += 1

    # Lead prioritization (if in focus areas)
    if "lead_prioritization" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "lead_prioritization",
            "description": "Rank and prioritize leads based on intent, engagement, risk, and urgency",
            "dependencies": ["data_loading"],
            "outputs": ["prioritized_leads", "top_priority_leads"]
        })
        step_num += 1

    # Customer needs analysis (if in focus areas)
    if "customer_needs_analysis" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "customer_needs_analysis",
            "description": "Analyze customer pain points, buying signals, and objection likelihood",
            "dependencies": ["data_loading"],
            "outputs": ["customer_needs_analysis"]
        })
        step_num += 1

    # Outreach generation (if in focus areas)
    if "outreach_generation" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "outreach_generation",
            "description": "Generate personalized outreach recommendations with message drafts and timing",
            "dependencies": ["lead_prioritization", "customer_needs_analysis"],
            "outputs": ["outreach_recommendations"]
        })
        step_num += 1

    # Follow-up coordination (if in focus areas)
    if "follow_up_coordination" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "follow_up_coordination",
            "description": "Identify and track follow-up actions, detect overdue items",
            "dependencies": ["data_loading"],
            "outputs": ["follow_up_actions"]
        })
        step_num += 1

    # Rep nudging (if in focus areas)
    if "rep_nudging" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "rep_nudging",
            "description": "Generate nudges for sales reps based on overdue follow-ups, stalled deals, and high-priority leads",
            "dependencies": ["follow_up_coordination", "lead_prioritization", "deal_insights"],
            "outputs": ["rep_nudges"]
        })
        step_num += 1

    # Deal insights (if in focus areas)
    if "deal_insights" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "deal_insights",
            "description": "Detect stalled deals, at-risk deals, and opportunities",
            "dependencies": ["data_loading"],
            "outputs": ["deal_insights", "stalled_deals", "at_risk_deals"]
        })
        step_num += 1

    # Historical insights (if in focus areas)
    if "historical_insights" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "historical_insights",
            "description": "Analyze win/loss patterns from past deals to surface actionable insights",
            "dependencies": ["data_loading"],
            "outputs": ["win_patterns", "loss_patterns"]
        })
        step_num += 1

    # Executive reporting (if in focus areas)
    if "executive_reporting" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "executive_reporting",
            "description": "Generate pipeline summary and rep performance metrics",
            "dependencies": ["deal_insights", "lead_prioritization"],
            "outputs": ["pipeline_summary", "rep_performance_summary"]
        })
        step_num += 1

    # Always end with report generation
    plan.append({
        "step": step_num,
        "name": "report_generation",
        "description": "Generate final markdown report with all insights and recommendations",
        "dependencies": [p["name"] for p in plan if p["name"] != "report_generation"],
        "outputs": ["enablement_report", "report_file_path"]
    })

    return {
        "plan": plan,
        "errors": state.get("errors", [])
    }


def data_loading_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Data Loading Node: Orchestrate loading all data files and creating lookups.

    This node loads:
    - Leads
    - Sales Reps
    - Interactions
    - Deals
    - Signals

    And creates lookup dictionaries for fast access.
    """
    errors = state.get("errors", [])
    config = SalesEnablementOrchestratorConfig()

    try:
        # Load all data files
        leads = load_leads(config.data_dir, config.leads_file)
        sales_reps = load_sales_reps(config.data_dir, config.sales_reps_file)
        interactions = load_interactions(config.data_dir, config.interactions_file)
        deals = load_deals(config.data_dir, config.deals_file)
        signals = load_signals(config.data_dir, config.signals_file)

        # Build lookup dictionaries
        leads_lookup = build_leads_lookup(leads)
        reps_lookup = build_reps_lookup(sales_reps)
        interactions_lookup = build_interactions_lookup(interactions)
        deals_lookup = build_deals_lookup(deals)
        signals_lookup = build_signals_lookup(signals)

        return {
            "leads": leads,
            "sales_reps": sales_reps,
            "interactions": interactions,
            "deals": deals,
            "signals": signals,
            "leads_lookup": leads_lookup,
            "reps_lookup": reps_lookup,
            "interactions_lookup": interactions_lookup,
            "deals_lookup": deals_lookup,
            "signals_lookup": signals_lookup,
            "errors": errors
        }
    except FileNotFoundError as e:
        return {
            "errors": errors + [f"data_loading_node: File not found - {str(e)}"]
        }
    except json.JSONDecodeError as e:
        return {
            "errors": errors + [f"data_loading_node: Invalid JSON - {str(e)}"]
        }
    except Exception as e:
        return {
            "errors": errors + [f"data_loading_node: Unexpected error - {str(e)}"]
        }


def lead_prioritization_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Lead Prioritization Node: Rank and prioritize leads.

    Calculates priority scores for all leads and selects top N.
    """
    errors = state.get("errors", [])
    leads = state.get("leads")
    signals_lookup = state.get("signals_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    if not leads:
        return {
            "errors": errors + ["lead_prioritization_node: leads required"]
        }

    try:
        # Prioritize all leads
        prioritized_leads = prioritize_leads(
            leads,
            signals_lookup,
            weights=config.priority_scoring_weights
        )

        # Get top N leads
        top_priority_leads = get_top_priority_leads(
            prioritized_leads,
            top_n=config.top_n_leads
        )

        return {
            "prioritized_leads": prioritized_leads,
            "top_priority_leads": top_priority_leads,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"lead_prioritization_node: {str(e)}"]
        }


def customer_needs_analysis_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Customer Needs Analysis Node: Analyze customer needs and pain points.

    Analyzes pain points, buying signals, objection likelihood, and product fit.
    """
    errors = state.get("errors", [])
    leads = state.get("leads")
    interactions_lookup = state.get("interactions_lookup", {})
    deals_lookup = state.get("deals_lookup", {})

    if not leads:
        return {
            "errors": errors + ["customer_needs_analysis_node: leads required"]
        }

    try:
        # Analyze all customer needs
        customer_needs_analysis = analyze_all_customer_needs(
            leads,
            interactions_lookup,
            deals_lookup
        )

        return {
            "customer_needs_analysis": customer_needs_analysis,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"customer_needs_analysis_node: {str(e)}"]
        }



# Test Phase 1: Foundation Nodes (Goal & Planning)

In [ ]:
"""Test Phase 1: Foundation Nodes (Goal & Planning)

Test goal_node and planning_node independently.
"""

from agents.sales_enablement.nodes import goal_node, planning_node
from config import SalesEnablementOrchestratorState


def test_goal_node():
    """Test goal_node with no input"""
    print("Testing goal_node...")

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = goal_node(state)

    assert "goal" in result, "goal_node should return 'goal'"
    assert result["goal"]["objective"] is not None, "goal should have objective"
    assert len(result["goal"]["focus_areas"]) > 0, "goal should have focus_areas"
    assert result["goal"]["scope"] == "all_leads", "scope should be 'all_leads' when no lead_id"

    print(f"✅ goal_node test passed!")
    print(f"   Objective: {result['goal']['objective'][:60]}...")
    print(f"   Focus areas: {len(result['goal']['focus_areas'])} areas")
    print(f"   Scope: {result['goal']['scope']}")
    print()


def test_goal_node_with_lead_id():
    """Test goal_node with specific lead_id"""
    print("Testing goal_node with lead_id...")

    state: SalesEnablementOrchestratorState = {
        "lead_id": "L-001",
        "errors": []
    }

    result = goal_node(state)

    assert result["goal"]["lead_id"] == "L-001", "goal should include lead_id"
    assert result["goal"]["scope"] == "single_lead", "scope should be 'single_lead'"

    print(f"✅ goal_node with lead_id test passed!")
    print(f"   Lead ID: {result['goal']['lead_id']}")
    print(f"   Scope: {result['goal']['scope']}")
    print()


def test_goal_node_with_focus_area():
    """Test goal_node with specific focus_area"""
    print("Testing goal_node with focus_area...")

    state: SalesEnablementOrchestratorState = {
        "focus_area": "lead_prioritization",
        "errors": []
    }

    result = goal_node(state)

    assert result["goal"]["focus_areas"] == ["lead_prioritization"], "focus_areas should match input"

    print(f"✅ goal_node with focus_area test passed!")
    print(f"   Focus areas: {result['goal']['focus_areas']}")
    print()


def test_planning_node():
    """Test planning_node with goal"""
    print("Testing planning_node...")

    state: SalesEnablementOrchestratorState = {
        "goal": {
            "objective": "Test objective",
            "focus_areas": ["lead_prioritization", "outreach_generation"]
        },
        "errors": []
    }

    result = planning_node(state)

    assert "plan" in result, "planning_node should return 'plan'"
    assert len(result["plan"]) > 0, "plan should have steps"

    # Check that plan includes data_loading (always first)
    assert result["plan"][0]["name"] == "data_loading", "First step should be data_loading"

    # Check that plan includes requested focus areas
    plan_names = [step["name"] for step in result["plan"]]
    assert "lead_prioritization" in plan_names, "plan should include lead_prioritization"
    assert "outreach_generation" in plan_names, "plan should include outreach_generation"

    # Check that plan ends with report_generation
    assert result["plan"][-1]["name"] == "report_generation", "Last step should be report_generation"

    print(f"✅ planning_node test passed!")
    print(f"   Total steps: {len(result['plan'])}")
    print(f"   Steps: {[step['name'] for step in result['plan']]}")
    print()


def test_planning_node_without_goal():
    """Test planning_node error handling when goal is missing"""
    print("Testing planning_node error handling...")

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = planning_node(state)

    assert "errors" in result, "planning_node should return errors"
    assert len(result["errors"]) > 0, "should have error when goal is missing"
    assert "goal is required" in result["errors"][0], "error should mention goal"

    print(f"✅ planning_node error handling test passed!")
    print(f"   Error: {result['errors'][0]}")
    print()


def test_goal_and_planning_chain():
    """Test goal_node → planning_node chain"""
    print("Testing goal_node → planning_node chain...")

    # Start with empty state
    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    # Run goal_node (merge result into state)
    goal_result = goal_node(state)
    state.update(goal_result)
    assert "goal" in state, "goal should be in state after goal_node"

    # Run planning_node (merge result into state)
    plan_result = planning_node(state)
    state.update(plan_result)
    assert "plan" in state, "plan should be in state after planning_node"
    assert len(state["errors"]) == 0, "should have no errors in successful chain"

    print(f"✅ goal_node → planning_node chain test passed!")
    print(f"   Goal objective: {state['goal']['objective'][:50]}...")
    print(f"   Plan steps: {len(state['plan'])}")
    print()


def test_data_loading_node():
    """Test data_loading_node"""
    print("Testing data_loading_node...")

    from agents.sales_enablement.nodes import data_loading_node

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = data_loading_node(state)

    assert "leads" in result, "data_loading_node should return 'leads'"
    assert "sales_reps" in result, "data_loading_node should return 'sales_reps'"
    assert "interactions" in result, "data_loading_node should return 'interactions'"
    assert "deals" in result, "data_loading_node should return 'deals'"
    assert "signals" in result, "data_loading_node should return 'signals'"
    assert "leads_lookup" in result, "data_loading_node should return 'leads_lookup'"
    assert "reps_lookup" in result, "data_loading_node should return 'reps_lookup'"
    assert "interactions_lookup" in result, "data_loading_node should return 'interactions_lookup'"
    assert "deals_lookup" in result, "data_loading_node should return 'deals_lookup'"
    assert "signals_lookup" in result, "data_loading_node should return 'signals_lookup'"
    assert len(result["errors"]) == 0, "should have no errors"

    # Verify data loaded correctly
    assert len(result["leads"]) == 20, "should load 20 leads"
    assert len(result["sales_reps"]) == 4, "should load 4 reps"
    assert len(result["interactions"]) == 12, "should load 12 interactions"
    assert len(result["deals"]) == 15, "should load 15 deals"
    assert len(result["signals"]) == 20, "should load 20 signals"

    # Verify lookups work
    assert "L-001" in result["leads_lookup"], "leads_lookup should contain L-001"
    assert "SR-01" in result["reps_lookup"], "reps_lookup should contain SR-01"

    print(f"✅ data_loading_node test passed!")
    print(f"   Loaded: {len(result['leads'])} leads, {len(result['sales_reps'])} reps, {len(result['interactions'])} interactions, {len(result['deals'])} deals, {len(result['signals'])} signals")
    print(f"   Lookups: {len(result['leads_lookup'])} leads, {len(result['reps_lookup'])} reps, {len(result['interactions_lookup'])} leads with interactions, {len(result['deals_lookup'])} leads with deals")
    print()

    return result


def test_lead_prioritization_node():
    """Test lead_prioritization_node"""
    print("Testing lead_prioritization_node...")

    from agents.sales_enablement.nodes import lead_prioritization_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then prioritize
    result = lead_prioritization_node(state)

    assert "prioritized_leads" in result, "should return prioritized_leads"
    assert "top_priority_leads" in result, "should return top_priority_leads"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["prioritized_leads"]) == 20, "should prioritize all 20 leads"
    assert len(result["top_priority_leads"]) == 10, "should return top 10 leads"
    assert result["prioritized_leads"][0]["priority_score"] >= result["prioritized_leads"][-1]["priority_score"], "should be sorted"

    print(f"✅ lead_prioritization_node test passed!")
    print(f"   Prioritized: {len(result['prioritized_leads'])} leads")
    print(f"   Top {len(result['top_priority_leads'])} leads:")
    for i, lead in enumerate(result["top_priority_leads"][:3], 1):
        print(f"     {i}. {lead['lead_id']} - {lead['company_name']} (Score: {lead['priority_score']:.2f})")
    print()


def test_customer_needs_analysis_node():
    """Test customer_needs_analysis_node"""
    print("Testing customer_needs_analysis_node...")

    from agents.sales_enablement.nodes import customer_needs_analysis_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then analyze
    result = customer_needs_analysis_node(state)

    assert "customer_needs_analysis" in result, "should return customer_needs_analysis"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["customer_needs_analysis"]) == 20, "should analyze all 20 leads"
    assert "buying_signals" in result["customer_needs_analysis"][0], "should have buying_signals"
    assert "product_fit" in result["customer_needs_analysis"][0], "should have product_fit"

    print(f"✅ customer_needs_analysis_node test passed!")
    print(f"   Analyzed: {len(result['customer_needs_analysis'])} leads")
    sample = result["customer_needs_analysis"][0]
    print(f"   Sample: {sample['lead_id']} - Product fit: {sample['product_fit']}, Buying signals: {len(sample['buying_signals'])}")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 1, 2 & 3: Foundation + Data Loading + Analysis Nodes - Test Suite")
    print("=" * 60)
    print()

    test_goal_node()
    test_goal_node_with_lead_id()
    test_goal_node_with_focus_area()
    test_planning_node()
    test_planning_node_without_goal()
    test_goal_and_planning_chain()
    test_data_loading_node()
    test_lead_prioritization_node()
    test_customer_needs_analysis_node()

    print("=" * 60)
    print("✅ All Phase 1, 2 & 3 tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_007_TEMPLATE copy % python test_sales_enablement_phase1.py
============================================================
Phase 1, 2 & 3: Foundation + Data Loading + Analysis Nodes - Test Suite
============================================================

Testing goal_node...
✅ goal_node test passed!
   Objective: Enable sales team performance by prioritizing leads, analyzi...
   Focus areas: 8 areas
   Scope: all_leads

Testing goal_node with lead_id...
✅ goal_node with lead_id test passed!
   Lead ID: L-001
   Scope: single_lead

Testing goal_node with focus_area...
✅ goal_node with focus_area test passed!
   Focus areas: ['lead_prioritization']

Testing planning_node...
✅ planning_node test passed!
   Total steps: 4
   Steps: ['data_loading', 'lead_prioritization', 'outreach_generation', 'report_generation']

Testing planning_node error handling...
✅ planning_node error handling test passed!
   Error: planning_node: goal is required

Testing goal_node → planning_node chain...
✅ goal_node → planning_node chain test passed!
   Goal objective: Enable sales team performance by prioritizing lead...
   Plan steps: 10

Testing data_loading_node...
✅ data_loading_node test passed!
   Loaded: 20 leads, 4 reps, 12 interactions, 15 deals, 20 signals
   Lookups: 20 leads, 4 reps, 9 leads with interactions, 15 leads with deals

Testing lead_prioritization_node...
✅ lead_prioritization_node test passed!
   Prioritized: 20 leads
   Top 10 leads:
     1. L-020 - Orion Aerospace (Score: 86.60)
     2. L-003 - Apex Manufacturing (Score: 85.50)
     3. L-011 - Atlas Freight (Score: 83.00)

Testing customer_needs_analysis_node...
✅ customer_needs_analysis_node test passed!
   Analyzed: 20 leads
   Sample: L-001 - Product fit: 0.63, Buying signals: 1

============================================================
✅ All Phase 1, 2 & 3 tests passed!
============================================================
